# Demonstration for using xhycom
**Author: Jun Sasaki, Coded on August 20, 2020, Updated on August 20, 2020**

In [ ]:
import xhycom.utils as xh
from xhycom.utils import PlotConfig, Data, Plotter
import xarray as xr
from netCDF4 import Dataset
from IPython.display import HTML, display
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.io.img_tiles import Stamen
from cartopy.io.img_tiles import OSM
import hvplot.xarray
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

## Downloading and creating multi-time netcdf
- `xh.run_opendap(extent, time_start, time_end=None, dtime=3, tz='utc')`
- Netcdf file name is automatically generated as "hycom_" + datetime + ".nc" 
- This method should be used in general.

In [ ]:
#extent = (139.2, 140.2, 34.8, 35.8)  ### Mouth of Tokyo Bay
extent = (-82.0, -65.0, 28.0, 43.0)  ### Project Sandy
time_start, time_end, dtime = ('2012-10-28 12:00:00', '2012-10-29 00:00:00', 3)  ### dtime (int): time interval in hours
### Reading catalog from local
#xh.run_opendap(extent=extent, time_start=time_start, time_end=time_end, dtime=dtime)
### Reading catalog from OPeNDAP
dirpath = 'http://tds.hycom.org/thredds/dodsC/GLBv0.08/expt_53.X/data/'
xh.run_opendap(extent=extent, time_start=time_start, time_end=time_end, dtime=dtime, opendap=True, dirpath=dirpath)

# Loading netcdf time series files into xarray.Dataset

In [ ]:
ncfiles = 'hycom_lon-82.0lon-65.0_lat28.0lat43.0_2012*.nc'

with xr.open_mfdataset(ncfiles, parallel=True, concat_dim="time", data_vars='minimal', \
                       coords='minimal', compat='override') as dsts:
    print("Creating xarray.Dataset from " + str(ncfiles))
dsts

# 2D Plotting for plan view, vertical views, and timeseries
Save method may be overridden by the following. You may put some text by self.fig.text(x, y, str) where x and y are normilized axes (0 and 1) and str is a plotted string. 

In [ ]:
### Specify plot type and variable
plot_type = 'plot_xy'
var = 'salinity' #'water_temp'

'''
### Tokyo Bay
### Speciry axes range or None
#extent_xy = (139.2, 140.2, 34.8, 35.8)
extent_xy = (139.0, 140.4, 34.6, 36.0)
extent_xz = (139.0, 140.4, 0, 3000) # None
extent_yz = None #(139.0, 140.4, 0, 3000)
'''
extent_tx = None
extent_ty = None
extent_tz = None

### Sandy
extent_xy = (-82.0, -65.0, 28.0, 43.0)
extent_xz = (-82.0, -65.0, 0, 5000.0)
extent_yz = None


### Specify axes' ticks intervals or None
ticks_intervals_xy = None  ### (0.4, 0.2)
ticks_intervals_xz = None
ticks_intervals_yz = None
ticks_intervals_tx = None
ticks_intervals_ty = None
ticks_intervals_tz = None

### Create Data instance and PlotConfig instance
da_var = Data(da=dsts[var])
cfg_sal = PlotConfig()

### Create Plotter instance and invoke its method of save or plot

vmin = None  ### 34.0
vmax = None  ### 34.2
levels = None ### 10

if   plot_type == 'plot_xy':
    ### xy plot (lon, lat)
    plot_xy=Plotter(plot_config=cfg_sal, data=da_var, x='lon', y='lat', z=0, t=0)
    plot_xy.plot(extent=extent_xy, ticks_intervals=ticks_intervals_xy, \
                 vmin=vmin, vmax=vmax, levels=10).save("./class_text_xy.png", dpi=300, bbox_inches='tight')
    #plot_xy.save("./class_test_xy.png", extent=extent_xy, ticks_intervals=ticks_intervals_xy, \
    #             vmin=vmin, vmax=vmax, levels=10, dpi=300, bbox_inches='tight')
elif plot_type == 'plot_xz':
    ### xz plot (lon, depth)
    plot_xz=Plotter(plot_config=cfg_sal, data=da_var, x='lon', y=0, z='depth', t=0)
    plot_xz.save("./class_test_xz.png", extent=extent_xz, ticks_intervals=ticks_intervals_xz, dpi=300, bbox_inches='tight')
elif plot_type == 'plot_yz':
    ### yz plot (lat, depth)
    plot_yz=Plotter(plot_config=cfg_sal, data=da_var, x=2, y='lat', z='depth', t=0)
    plot_yz.save("./class_test_yz.png", extent=extent_yz, ticks_intervals=ticks_intervals_yz, dpi=300, bbox_inches='tight')
elif plot_type == 'plot_tx':
    ### tx plot (time, lon)
    plot_tx=Plotter(plot_config=cfg_sal, data=da_var, x='lon', y=0, z=0, t='time')
    plot_tx.save("./class_test_tx.png", extent=extent_tx, ticks_intervals=ticks_intervals_tx, dpi=300, bbox_inches='tight')
elif plot_type == 'plot_ty':
    ### ty plot (time, lat)
    plot_ty=Plotter(plot_config=cfg_sal, data=da_var, x=0, y='lat', z=0, t='time')
    plot_ty.save("./class_test_ty.png", extent=extent_ty, ticks_intervals=ticks_intervals_ty, dpi=300, bbox_inches='tight')
elif plot_type == 'plot_tz':
    ### tz plot (time, depth)
    plot_tz=Plotter(plot_config=cfg_sal, data=da_var, x=0, y=0, z='depth', t='time')
    plot_tz.save("./class_test_tz.png", extent=extent_tz, ticks_intervals=ticks_intervals_tz, dpi=300, bbox_inches='tight')
else:
    'ERROR: No such plot_type'

# Creating GIF animation
Only supported plot_type of `plot_xy`. It is recommended to prepare a specific manual code for animation without using class because cutomizing panels is often required.

In [ ]:
gif_file = "./test_ani.gif"
plot_type = 'plot_xy'
var = 'salinity' #'water_temp'
extent_xy = (139.0, 140.4, 34.6, 36.0)
ticks_intervals_xy = (0.4, 0.2)
### Create Data instance and PlotConfig instance
da_var = Data(da=dsts[var])
cfg_sal = PlotConfig()

if plot_type == 'plot_xy':
    ### xy plot (lon, lat)
    plot_xy=Plotter(plot_config=cfg_sal, data=da_var, x='lon', y='lat', z=0, t=slice(None))

    ani = plot_xy.frame(extent=extent_xy, ticks_intervals=ticks_intervals_xy, vmin=34.0, vmax=34.2, levels=20, \
                  cbar_kwargs={'shrink':0.8}).anim(gif_file, frames=None, \
                                                   interval=500, writer='pillow', dpi=200)

## Display GIF animation

In [ ]:
display(HTML(ani.to_jshtml()))

## Create and display MP4 animation
Instance of animation is converted to MP4 animation.

In [ ]:
#mp4_file = "./test_ani.mp4"
#ani.save(mp4_file, dpi=100)
#display(HTML("<video controls><source src='./test_ani.mp4' type='video/mp4'></video>")) ### File name should be in '***'

In [ ]:
mp4_file = "./test_ani.mp4"
plot_xy=Plotter(plot_config=cfg_sal, data=da_var, x='lon', y='lat', z=0, t=slice(None))

ani_mp4 = plot_xy.frame(extent=extent_xy, ticks_intervals=ticks_intervals_xy, vmin=34.0, vmax=34.2, levels=20, \
          cbar_kwargs={'shrink':0.8}).anim(gif_file, frames=None, \
                       interval=500, writer='ffmpeg', dpi=200)
ani_mp4.save(mp4_file, dpi=100)
display(HTML("<video controls><source src='./test_ani.mp4' type='video/mp4'></video>"))

# Interactive plotting with [hvPlot](https://hvplot.holoviz.org/#)
- Useful for checking data values of `(x, y, z)` interactively.
- Slower than matplotlib; may not be suitable for publication quality plots.
- [Geoviews](https://geoviews.org/#) may also be useful.

[hvPlot geographic data](https://hvplot.holoviz.org/user_guide/Geographic_Data.html) and [hvPlot options](https://hvplot.holoviz.org/user_guide/Customization.html)<br>

## Plan view
`xarray.Dataset.hvplot.quadmesh(x, y, z, project, coastline, frame_height, cmap)`<br>
- [cmap](https://matplotlib.org/gallery/color/colormap_reference.html?highlight=cmap): matplotlib colormap (for details see [here](https://matplotlib.org/tutorials/colors/colormaps.html))
- `coastline = ['10m'/'50m'/'110m']`
- When `tiles` is specified (e.g., `tiles="OSM"`), axes labels are ignored -> maybe a bug.


In [ ]:
### When tiles is not None (e.g., tiles="OSM"), axes labels are to be ignored -> maybe bug.
z='salinity'
time = 0  ### by index
cmap='magma_r' ## _r : reversed 
frame_height=300
project = ccrs.PlateCarree()
dsts.isel(time=time).hvplot.quadmesh(x='lon', y='lat', z=z, project=project, tiles=None, \
                                     coastline='10m', frame_height=frame_height, cmap=cmap)

## Vertical sectional views
`xarray.Dataset.hvplot.quadmesh(x, y, z, flip_yaxis, frame_width, frame_height, cmap)`<br>
- `flip_yaxis=True`: reverse yaxis

In [ ]:
time = 0
x = 'lon'
z = 'salinity'
cmap = 'magma_r'
frame_width = 300
frame_height = 200

dsts.isel(time=time).hvplot.quadmesh(x=x, y='depth', z=z, flip_yaxis=True, \
                                     frame_width=frame_width, frame_height=frame_height, cmap=cmap)

In [ ]:
time = 0
x = 'lat'  ### x = 'lat'|'lon'

dsts.isel(time=time).hvplot.quadmesh(x=x, y='depth', z=z, flip_yaxis=True, \
                                     frame_width=frame_width, frame_height=frame_height, cmap=cmap)

# Interactive time series plotting with hvPlot
- Using dask for large data handling
- See [Readng and writing files - xarray](http://xarray.pydata.org/en/stable/io.html)

## Plan view
`xarray.Dataset.hvplot.quadmesh(x, y, z, project, coastline, frame_height, cmap)`

In [ ]:
z = 'salinity'
project = ccrs.PlateCarree()
frame_height = 300
cmap = 'magma_r'

dsts.hvplot.quadmesh(x='lon', y='lat', z=z, project=project, coastline='10m', \
                     frame_height=frame_height, cmap=cmap)

## Vertical sectional views

In [ ]:
x = 'lon'
z = 'salinity'
frame_width  = 300
frame_height = 200
cmap = 'magma_r'

dsts.hvplot.quadmesh(x=x, y='depth', z=z, flip_yaxis=True, frame_height=frame_height, frame_width=frame_width, \
                     cmap=cmap)

In [ ]:
x = 'lat'
dsts.hvplot.quadmesh(x=x, y='depth', z=z, flip_yaxis=True, frame_height=frame_height, frame_width=frame_width, \
                     cmap=cmap)